### Importing necessary modules 

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime

### Using Alpha Vantage API to collect stock data of Microsoft till date

In [2]:
df_msft=web.DataReader("MSFT","av-daily",start=datetime(1995,1,1),end=datetime.now(),api_key="3BB05VMWE1JEX67I0")

In [3]:
df_msft.head()

,open,high,low,close,volume
1999-11-01,93.25,94.19,92.12,92.37,26630600
1999-11-02,92.75,94.50,91.94,92.56,23174500
1999-11-03,92.94,93.50,91.50,92.00,22258500
1999-11-04,92.31,92.75,90.31,91.75,27119700
1999-11-05,91.81,92.87,90.50,91.56,35083700


### Cleaning data
* We do not need high , low and volume columns for our project
* We need to drop any rows with missing values 
* Fill empty places which are necessary for calculations

In [4]:
#dropping unnecessary columns
df_msft.drop(["high","low","volume"],axis=1,inplace=True)

In [5]:
df_msft.columns

Index(['open', 'close'], dtype='object')

In [6]:
#dropping rows with missing values
df_msft.dropna(how='all')

,open,close
1999-11-01,93.25,92.37
1999-11-02,92.75,92.56
1999-11-03,92.94,92.00
1999-11-04,92.31,91.75
1999-11-05,91.81,91.56
...,...,...
2020-10-01,213.49,212.46
2020-10-02,208.00,206.19
2020-10-05,207.22,210.38
2020-10-06,208.82,205.91


In [7]:
#filling single cell with missing values with 'nan' so that those cells can be avoided during calculaions such as count
df_msft.replace([" ","missing"],np.nan)
#using "inplace=True" is recommended to make changes permanant

,open,close
1999-11-01,93.25,92.37
1999-11-02,92.75,92.56
1999-11-03,92.94,92.00
1999-11-04,92.31,91.75
1999-11-05,91.81,91.56
...,...,...
2020-10-01,213.49,212.46
2020-10-02,208.00,206.19
2020-10-05,207.22,210.38
2020-10-06,208.82,205.91


### Applying aggregations and resampling based on date-time index

In [8]:
# calculating daily growth based on [growth=close-open]
df_msft["growth"]=df_msft["close"]-df_msft["open"]

In [9]:
df_msft.head()

,open,close,growth
1999-11-01,93.25,92.37,-0.88
1999-11-02,92.75,92.56,-0.19
1999-11-03,92.94,92.00,-0.94
1999-11-04,92.31,91.75,-0.56
1999-11-05,91.81,91.56,-0.25


In [10]:
#transforming the index into date-time series 
df_msft.index=pd.to_datetime(df_msft.index)

In [11]:
#scraping month and year from date-time index 
df_msft["month"]=[i.month for i in df_msft.index]
df_msft["year"]=[i.year for i in df_msft.index]    

In [12]:
grwth_year=df_msft.groupby(["year"]).growth.mean()

In [13]:
grwth_year=round(grwth_year,2)

In [14]:
#average growth per year 
grwth_year

year
1999    0.64
2000   -0.21
2001    0.15
2002   -0.03
2003   -0.04
2004   -0.01
2005   -0.00
2006    0.03
2007    0.01
2008   -0.04
2009    0.04
2010   -0.01
2011    0.01
2012   -0.01
2013    0.05
2014    0.02
2015    0.05
2016    0.03
2017    0.03
2018   -0.09
2019    0.04
2020    0.14
Name: growth, dtype: float64

In [15]:
#calculating highest and lowest price achieved in years
high_year=df_msft.groupby(["year"]).max().close
low_year=df_msft.groupby(["year"]).min().close

In [16]:
#average stock price per year
avg_stck_year=df_msft.groupby(["year"]).mean().close

In [17]:
avg_stck_year

year
1999     98.020465
2000     76.219603
2001     62.542460
2002     54.549048
2003     29.238214
2004     27.124718
2005     25.871031
2006     26.290792
2007     30.445876
2008     26.647508
2009     22.976556
2010     27.058353
2011     26.052157
2012     29.820274
2013     32.491389
2014     42.453343
2015     46.713571
2016     55.259306
2017     71.984024
2018    101.033984
2019    130.382024
2020    186.179485
Name: close, dtype: float64

### Combining all processed data and writing out to file

In [23]:
# Combining using pandas concat function
df_processed=pd.concat([grwth_year,high_year,low_year,avg_stck_year],axis=1,ignore_index=True)

In [25]:
#renaming columns with meaningfull labels
df_processed.rename(columns={0:'growth',1:'high',2:'low',3:'average price'},inplace=True)

In [26]:
df_processed

,growth,high,low,average price
year,,,,
1999,0.64,119.12,84.940,98.020465
2000,-0.21,116.56,41.500,76.219603
2001,0.15,73.68,43.380,62.542460
2002,-0.03,69.86,42.830,54.549048
2003,-0.04,56.97,22.800,29.238214
2004,-0.01,29.98,24.150,27.124718
2005,-0.00,28.16,23.920,25.871031
2006,0.03,30.19,21.510,26.290792
2007,0.01,37.06,26.720,30.445876


In [28]:
#writing out data to csv for furthur process
df_processed.to_csv("data_for_visualization.csv")